### Simple test notebook
This notebook is a simple test of the functionality of the SOCS method with a very small dataset, to ensure that the software works properly.

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.io as io
from scipy import stats
import scanpy as sc
import anndata as ad
import squidpy as sq
import copy
import seaborn as sns
import pickle
import math
import sklearn
import torch
import socs
from socs.utils import row_normalize
from socs.analysis import map_vector_sampled,hillNumber_1,vec2vec

//broad/clearylab/Users/Peter/anaconda3/envs/myEnv/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED
2025-02-18 15:26:26.955525: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-02-18 15:26:26.984930: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-02-18 15:26:27.019373: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been regist

We'll load in a very small sample from the two timepoints in the MERFISH mouse ovary datasets.

In [12]:
adata_1 = sc.read_h5ad('example_data/adata_1_follicle.h5ad')
adata_2 = sc.read_h5ad('example_data/adata_2_follicle.h5ad')
adata_1.obs['time'] = 0
adata_2.obs['time'] = 1

We'll concatenate these two datasets, and perform principal component analysis on the combined object's gene expression count table. 

In [13]:
adata_concat= ad.concat([adata_1,adata_2])
sc.pp.pca(adata_concat, random_state= 0,n_comps=30)
adata_concat_1 = adata_concat[0:adata_1.shape[0],:]
adata_concat_2 = adata_concat[adata_1.shape[0]:,:]

//broad/clearylab/Users/Peter/anaconda3/envs/myEnv/lib/python3.9/site-packages/anndata/_core/anndata.py:1906: UserWarning: Observation names are not unique. To make them unique, call `.obs_names_make_unique`.
  utils.warn_names_duplicates("obs")


Then, we'll set up the SOCS problem using the class SOCSModel, using a set of parameters $\alpha$, $\epsilon$, $\rho_1$, and $\rho_2$, as described in __Methods__

Finally, we'll use the SOCS algorithm to estimate the trajectory mapping $T$ between the samples at $t_1$ and $t_2$.

In [14]:
alpha = 0.5
eps = 2e-4
rho1 = 5000.0
rho2 = 5000.0
socs_model = socs.ot.SOCSModel(adata_concat,'time',expr_key='X_pca',struct_key='struct',alpha=alpha,eps=eps,rho=rho1,rho2=rho2)

We then use this SOCS problem to learn a transport map $T$, where element $(i,j)$ of $T$ is related to the probability that cell $i$ at $t_1$ maps to cell $j$ at $t_2$

In [15]:
T_socs = socs_model.infer_map(0,1)